# Disease Norm

In this example, we'll be writing an application to extract *mentions of* diseases from Pubmed abstracts, using annotations from the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial, which has 5 parts, walks through the process of constructing a model to classify _candidate_ disease mentions as either true (i.e., that it is truly a mention of a disease) or false.

## Part II: `Candidate` Extraction

In [ ]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

## Loading the `Corpus`

First, we will load the `Corpus` that we preprocessed in Part I:

In [ ]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'CDR Training').one()
corpus

Next, we collect each `Sentence` in the `Corpus` into a `set`.

In [ ]:
sentences = set()
for document in corpus:
    for sentence in document.sentences:
        sentences.add(sentence)

## Defining a `Candidate` schema
We now define the schema of the relation mention we want to extract (which is also the schema of the candidates).  This must be a subclass of `Candidate`, and we define it using a helper function.

Here we'll define a unary _disease relation mention_ which encapsulates a `Span` of text.  Note that this function will create the table in the database backend if it does not exist:

In [ ]:
from snorkel.models import candidate_subclass

Disease = candidate_subclass('Disease', ['disease'])

## Writing a basic NP `CandidateExtractor`

In [ ]:
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import RegexMatchSpan

ngrams = Ngrams(n_max=8)

# Define a noun phrase matcher
NP_RGX = r'^(((JJ|VBN|VBD|RB) )*((NN(P|S)?|POS) )*NN(P|S)?|JJ)$'
np_matcher = RegexMatchSpan(attrib='pos_tags', rgx=NP_RGX, longest_match_only=True)

ce = CandidateExtractor(Disease, [ngrams], [np_matcher])

## Running the `CandidateExtractor`

We run the `CandidateExtractor` by calling extract with the contexts to extract from, a name for the `CandidateSet` that will contain the results, and the current session.

In [ ]:
%time c = ce.extract(sentences, 'CDR Training Candidates', session)
print "Number of candidates:", len(c)

### Saving the extracted candidates

In [ ]:
session.add(c)
session.commit()

### Reloading the candidates

In [ ]:
from snorkel.models import CandidateSet
c = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Training Candidates').one()
c

## Using the `Viewer` to inspect candidates

Next, we'll use the `Viewer` class--here, specifically, the `SentenceNgramViewer`--to inspect the data.

It is important to note, our goal here is to **maximize the recall of true candidates** extracted, **not** to extract _only_ the correct candidates. Learning to distinguish true candidates from false candidates is covered in Tutorial 4.

First, we instantiate the `Viewer` object, which groups the input `Candidate` objects by `Sentence`:

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(c[:300], session, annotator_name="Tutorial Part 2 User")
else:
    sv = None

Next, we render the `Viewer.

In [ ]:
sv

Note that we can **navigate using the provided buttons**, or **using the keyboard (hover over buttons to see controls)**, highlight candidates (even if they overlap), and also **apply binary labels** (more on where to use this later!).  In particular, note that **the Viewer is synced dynamically with the notebook**, so that we can for example get the `Candidate` that is currently selected. Try it out!

In [ ]:
if 'CI' not in os.environ:
    print sv.get_selected()

### Repeating for development and test corpora
We will rerun the same operations for the other two CDR corpora: development and test. All we do for each is load in the `Corpus` object, collect the `Sentence` objects, and run them through the `CandidateExtractor`.

In [ ]:
for corpus_name in ['CDR Development', 'CDR Test']:
    corpus = session.query(Corpus).filter(Corpus.name == corpus_name).one()
    sentences = set()
    for document in corpus:
        for sentence in document.sentences:
            sentences.add(sentence)
    
    %time c = ce.extract(sentences, corpus_name + ' Candidates', session)
    session.add(c)
session.commit()

Next, in Part 3, we will annotate some candidates with labels so that we can evaluate performance.